# Preprocessing Halide Perovskite Dataset

1. 이 코드는 Nurion supercomputing server에서 계산된 **Nonlocal Halide Perovskite Dataset**에 대한 전처리 과정을 다룬다.
2. 이 코드는 Pymatgen.io.vasp.from_file() 메서드의 알 수 없는 에러로 인해 다른 google drive 계정에서 실행되었다. 그러므로 이 코드를 다시 실행하고자 한다면, 3600개의 서브디렉토리가 압축해제되어 있는 경로를 재설정해야 한다.
3. 전처리의 최종결과로, 3600개의 서브디렉토리에서 결정구조 VS Energy per atom 정보가 정리된 데이터가 pickle 형식으로 얻어진다.




## Google Drive Mount



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Hyeongseon Park Research/Halide Perovskite Dataset
%ls -l

/content/drive/My Drive/Hyeongseon Park Research/Halide Perovskite Dataset
total 295057
-rw------- 1 root root 302137490 Jan 17 06:10 nonlocal_perovskites_dataset_extracted.tar.gz


In [ ]:
# DFT Dataset 압축해제 안 된 경우에만 다음 코드를 실행!!
# !tar -zxvf nonlocal_perovskites_dataset_extracted.tar.gz

## CGCNN Github Repository Clone



In [ ]:
#!git clone https://github.com/txie-93/cgcnn

## Pymatgen Library Installation



In [ ]:
!pip install pymatgen==2020.11.11

In [ ]:
!pip list | grep imgaug
!pip list | grep pymatgen
!python --version

imgaug                        0.2.9
pymatgen                      2020.11.11
Python 3.7.12


## XML File Parsing



In [ ]:
%cd  /content/drive/MyDrive/Hyeongseon Park Research/Halide Perovskite Dataset/0001_CsPbBr3_mp-600089_1x1x1_with_noise_1
%ls -l

/content/drive/MyDrive/Hyeongseon Park Research/Halide Perovskite Dataset/0001_CsPbBr3_mp-600089_1x1x1_with_noise_1
total 494
-rw------- 1 root root    864 Jan 15 12:36 CONTCAR
-rw------- 1 root root   1005 Jan 15 12:36 INCAR
-rw------- 1 root root     48 Jan 15 12:36 KPOINTS
-rw------- 1 root root    275 Jan 15 12:36 POSCAR
-rw------- 1 root root 502193 Jan 15 12:36 vasprun.xml


In [ ]:
# Library Import for XML Parsing
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

In [ ]:
# 개별 Sub Directory에서 하나의 vasprun.xml 파일에 대해 분석을 시도하고
# 성공하면 다른 모든 디렉터리에 대해 적용한다.

# -------------------- 이 부분은 다른 모든 sub directory에서 공통적으로 정의되어야 한다. --------------------
xml_file = '/content/drive/MyDrive/Hyeongseon Park Research/Halide Perovskite Dataset/0001_CsPbBr3_mp-600089_1x1x1_with_noise_1/vasprun.xml'
doc = ET.parse(xml_file) # Parse XML document into element tree
root = doc.getroot()     # Return root element of this tree
# -------------------------------------------------------------------------------------------------------

print('XML Document : {}'.format(doc))
print('XML Root : {}'.format(root))
print('Root Node Name : {}'.format(root.tag))
print('Root Attributes : {}'.format(root.attrib))


XML Document : <xml.etree.ElementTree.ElementTree object at 0x7f53c4aa2e10>
XML Root : <Element 'modeling' at 0x7f53c4342bf0>
Root Node Name : modeling
Root Attributes : {}


In [ ]:
# 최상위노드(Root node)에 종속된 sub nodes 조사
for sub_node in root:
    print(sub_node.tag, sub_node.attrib) # 최상위노드 아래 서브노드의 노드명과 속성(딕셔너리) 출력

# vasprun.xml의 초기 노드정보는 DFT 시뮬레이션 조건을 명시하는 듯 하다.
# calculation 노드에 종속된 정보들이 아마도 DFT 수렴과정에서의 history 정보인 것 같다.

generator {}
incar {}
structure {'name': 'primitive_cell'}
varray {'name': 'primitive_index'}
kpoints {}
parameters {}
atominfo {}
structure {'name': 'initialpos'}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
calculation {}
structure {'name': 'finalpos'}


In [ ]:
print(root[0]) # root에 종속된 첫번째 'generator' 노드

<Element 'generator' at 0x7f53c4a2ad70>


In [ ]:
# XML file은 기본적으로 nested structure이므로 nested index로 접근가능하다. 이는 다차원 리스트 또는 다차원 배열과 유사
for i in range(6):                                            # generator의 하위태그가 6개 있다는 것을 이미 알고 있는 상태에서 확인
    print(root[0][i].tag, root[0][i].attrib, root[0][i].text) # Node name / Node Attributes / Node Contents 출력

i {'name': 'program', 'type': 'string'} vasp 
i {'name': 'version', 'type': 'string'} 5.4.4.18Apr17-6-g9f103f2a35  
i {'name': 'subversion', 'type': 'string'} (build Nov 05 2020 10:55:47) complex            parallel 
i {'name': 'platform', 'type': 'string'} LinuxIFC 
i {'name': 'date', 'type': 'string'} 2021 10 28 
i {'name': 'time', 'type': 'string'} 20:01:42 


In [ ]:
# 일단 XML file의 최상위노드와 종속되는 서브노드의 정보출력은 가능해졌다.
# Nested Index 구조를 통해 최상위노드 -> 하위노드에 이르기까지 모든 구조에 접근가능하다.
for i in range(len(root)):             # 최상위노드 root의 바로 밑의 서브노드의 수에 대해
    print(root[i].tag, root[i].attrib) # 1차 subnode name / attributes

    for j in range(len(root[i])):                      # 1차 서브노드 밑의 서브노드의 수에 대해
        print('\t', root[i][j].tag, root[i][j].attrib) # 2차 subnode name / attributes

        for k in range(len(root[i][j])):                           # 2차 서브노드 밑의 서브노드의 수에 대해
            print('\t\t', root[i][j][k].tag, root[i][j][k].attrib) # 3차 subnode name / attributes

            for m in range(len(root[i][j][k])):                                # 3차 서브노드 밑의 서브노드의 수에 대해
                print('\t\t\t', root[i][j][k][m].tag, root[i][j][k][m].attrib) # 4차 subnode name / attributes
    
    print('-----------------------------------------------------------------') # 1차 subnode 기준으로 구분

Streaming output truncated to the last 5000 lines.
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v {}
		 v 

In [ ]:
# 위와 같은 Nested Index 방식으로 XML file의 모든 정보에 접근가능하지만
# 이런 방식으로는 특정한 node name / attributes를 갖는 정보에 접근하기 불편하다.
# 따라서 node name 또는 node attributes 정보만으로 특정 정보에 접근할 수단이 필요하다. --> 이와 관련된 명령어 : find / findall / iter 
# find, findall : 의도대로 작동하지 않는다?
# iter : 특정 상위/하위 노드를 가리지 않고 모두 검색
# 나중에 test xml 파일로 find / findall / iter의 효과를 제대로 확인해 볼 것!

### 1.결정구조의 구성원자 정보 파악하기 From POSCAR


In [ ]:
!cat POSCAR

Cs1 Pb1 Br3
1.0
6.015318 0.006758 -0.016666
-0.006208 6.015856 0.004583
0.023221 -0.000894 6.001748
Cs Pb Br
1 1 3
direct
0.501881 0.487856 0.504101 Cs
0.004966 -0.010828 0.003167 Pb
-0.003399 0.013901 0.496070 Br
0.004248 0.504661 0.015057 Br
0.503523 0.005362 -0.002678 Br


In [ ]:
# 이상하게 내 Colab 환경에서는 작동되지 않는다. 다른 환경에서 전처리 후 데이터베이스화하여 사용해야 한다..
from pymatgen.io.vasp import Poscar
poscar = Poscar.from_file('POSCAR')
crystal = poscar.structure
crystal 

Structure Summary
Lattice
    abc : 6.015344883482907 6.015860948849217 6.00179298789795
 angles : 89.96511455647386 89.93707400045808 89.99487693420991
 volume : 217.18958188212017
      A : 6.015318 0.006758 -0.016666
      B : -0.006208 6.015856 0.004583
      C : 0.023221 -0.000894 6.001748
PeriodicSite: Cs (3.0277, 2.9378, 3.0194) [0.5019, 0.4879, 0.5041]
PeriodicSite: Pb (0.0300, -0.0651, 0.0189) [0.0050, -0.0108, 0.0032]
PeriodicSite: Br (-0.0090, 0.0832, 2.9774) [-0.0034, 0.0139, 0.4961]
PeriodicSite: Br (0.0228, 3.0360, 0.0926) [0.0042, 0.5047, 0.0151]
PeriodicSite: Br (3.0288, 0.0357, -0.0244) [0.5035, 0.0054, -0.0027]

In [ ]:
crystal.frac_coords # 속성으로 접근하면 Fractional Coodinates는 정상적으로 입력되어 있다.

array([[ 0.501881,  0.487856,  0.504101],
       [ 0.004966, -0.010828,  0.003167],
       [-0.003399,  0.013901,  0.49607 ],
       [ 0.004248,  0.504661,  0.015057],
       [ 0.503523,  0.005362, -0.002678]])

In [ ]:
crystal.lattice # Lattice Parameter

Lattice
    abc : 6.015344883482907 6.015860948849217 6.00179298789795
 angles : 89.96511455647386 89.93707400045808 89.99487693420991
 volume : 217.18958188212017
      A : 6.015318 0.006758 -0.016666
      B : -0.006208 6.015856 0.004583
      C : 0.023221 -0.000894 6.001748

In [ ]:
crystal.lattice.abc # A, B, C는 모두 정상입력! POSCAR를 정상적으로 사용가능하다!

(6.015344883482907, 6.015860948849217, 6.00179298789795)

### 2.결정구조의 Basis 변화 정보도 포함되어야 한다.



In [ ]:
# 다음과 같은 방식 : 각 단계의 서브노드에서 iter 명령을 사용하여 특정 노드에 접근하는 것이 가장 편리한 것 같다.
history_count = 0 # Calculation history 숫자와 일치하는지 확인하기 위해 count
for subnode1 in root.iter('calculation'):
    for subnode2 in subnode1.iter('structure'):
        for subnode3 in subnode2.iter('varray'):
            if subnode3.attrib == {'name' : 'basis'}:
                for i in range(len(subnode3)):
                    history_count += 1
                    print(subnode3[i].tag, subnode3[i].attrib, subnode3[i].text) # 관심정보의 node name / attributes / contents 출력
                print('-------------------------------------------------------')

print('calculation history : {}'.format(history_count)) # 일치 확인할 것!

v {}        6.01531800       0.00675800      -0.01666600 
v {}       -0.00620800       6.01585600       0.00458300 
v {}        0.02322100      -0.00089400       6.00174800 
-------------------------------------------------------
v {}        6.00969169       0.00675578      -0.01621760 
v {}       -0.00621177       6.02052999       0.00370127 
v {}        0.02365031      -0.00177465       6.00270322 
-------------------------------------------------------
v {}        5.99304902       0.00674921      -0.01488983 
v {}       -0.00622296       6.03438925       0.00109024 
v {}        0.02492167      -0.00438251       6.00555017 
-------------------------------------------------------
v {}        5.95984486       0.00673616      -0.01223441 
v {}       -0.00624542       6.06219107      -0.00413193 
v {}        0.02746480      -0.00959842       6.01132642 
-------------------------------------------------------
v {}        5.97391450       0.00674168      -0.01336063 
v {}       -0.00623588

In [ ]:
import re  # 정규표현식(Regular Expression)

total_basis = []
for subnode1 in root.iter('calculation'):
    for subnode2 in subnode1.iter('structure'):
        for subnode3 in subnode2.iter('varray'):
            if subnode3.attrib == {'name' : 'basis'}:
                each_basis = []
                for i in range(len(subnode3)):
                    basis = [float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)] # 정규표현식으로 숫자 변환
                    each_basis.append(basis)
                total_basis.append(each_basis)

In [ ]:
# 전처리 과정 확인
subnode3[i].text

'       0.00387861       0.03512720       0.50603244 '

In [ ]:
# 전처리 결과
[float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)]

[0.00387861, 0.0351272, 0.50603244]

In [ ]:
import numpy as np
total_basis = np.array(total_basis)
total_basis.shape # 50개의 계산 history가 있고, 그 각각에 대하여 3축 방향의 basis가 3개의 좌표로 표현된다.

(50, 3, 3)

### 3.원자들의 positions 정보 추출하기

* vasprun.xml file에서 positions 정보는 `<varray name = 'positions'>`로 표현된다.

In [ ]:
count = 0
for subnode1 in root.iter('calculation'):
    for subnode2 in subnode1.iter('structure'):
        for subnode3 in subnode2.iter('varray'):
            if subnode3.attrib == {'name' : 'positions'}:
                for i in range(len(subnode3)):
                    count += 1
                    print(subnode3[i].tag, subnode3[i].attrib, subnode3[i].text)
                print('-------------------------------------------------------')
print(count) # 250 : 50개의 history가 있고, 5개의 원자 좌표정보가 3개의 수로 표현된다.

v {}        0.50188100       0.48785600       0.50410100 
v {}        0.00496600       0.98917200       0.00316700 
v {}        0.99660100       0.01390100       0.49607000 
v {}        0.00424800       0.50466100       0.01505700 
v {}        0.50352300       0.00536200       0.99732200 
-------------------------------------------------------
v {}        0.50191782       0.48759038       0.50402188 
v {}        0.00478591       0.98999986       0.00303285 
v {}        0.99669232       0.01381489       0.49646876 
v {}        0.00429197       0.50422528       0.01494063 
v {}        0.50353098       0.00532159       0.99725287 
-------------------------------------------------------
v {}        0.50202696       0.48680439       0.50378751 
v {}        0.00425216       0.99244941       0.00263608 
v {}        0.99696292       0.01356027       0.49764936 
v {}        0.00442233       0.50293594       0.01459588 
v {}        0.50355463       0.00520198       0.99704817 
------------------

In [ ]:
# positions 정보에 접근하는 것에 성공했으므로 이제 전처리하여 추출한다.
total_positions = []
for subnode1 in root.iter('calculation'):
    for subnode2 in subnode1.iter('structure'):
        for subnode3 in subnode2.iter('varray'):
            if subnode3.attrib == {'name' : 'positions'}:
                
                each_positions = []
                for i in range(len(subnode3)):
                    pos = [float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)]
                    each_positions.append(pos)
                total_positions.append(each_positions)

In [ ]:
# 전처리 확인
subnode3[i].text

'       0.50346169       0.02973328       0.00439699 '

In [ ]:
# 전처리 결과
[float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)]

[0.50346169, 0.02973328, 0.00439699]

In [ ]:
total_positions = np.array(total_positions)
total_positions.shape # 50개의 history가 있고, 그 각각에 대해 5개의 구성원자를 3개의 좌표로 표현한다.

(50, 5, 3)

#### history positions 정보를 crystal.frac_coords에 삽입하기



In [ ]:
crystal # POSCAR로부터 읽은 계산 초기의 결정구조 정보

Structure Summary
Lattice
    abc : 6.015344883482907 6.015860948849217 6.00179298789795
 angles : 89.96511455647386 89.93707400045808 89.99487693420991
 volume : 217.18958188212017
      A : 6.015318 0.006758 -0.016666
      B : -0.006208 6.015856 0.004583
      C : 0.023221 -0.000894 6.001748
PeriodicSite: Cs (3.0277, 2.9378, 3.0194) [0.5019, 0.4879, 0.5041]
PeriodicSite: Pb (0.0300, -0.0651, 0.0189) [0.0050, -0.0108, 0.0032]
PeriodicSite: Br (-0.0090, 0.0832, 2.9774) [-0.0034, 0.0139, 0.4961]
PeriodicSite: Br (0.0228, 3.0360, 0.0926) [0.0042, 0.5047, 0.0151]
PeriodicSite: Br (3.0288, 0.0357, -0.0244) [0.5035, 0.0054, -0.0027]

In [ ]:
crystal.lattice # 격자 정보

Lattice
    abc : 6.015344883482907 6.015860948849217 6.00179298789795
 angles : 89.96511455647386 89.93707400045808 89.99487693420991
 volume : 217.18958188212017
      A : 6.015318 0.006758 -0.016666
      B : -0.006208 6.015856 0.004583
      C : 0.023221 -0.000894 6.001748

In [ ]:
crystal.lattice.matrix # 격자를 정의하는 행렬 정보

array([[ 6.015318e+00,  6.758000e-03, -1.666600e-02],
       [-6.208000e-03,  6.015856e+00,  4.583000e-03],
       [ 2.322100e-02, -8.940000e-04,  6.001748e+00]])

In [ ]:
from pymatgen.core.structure import Structure, Lattice

test_lattice = Lattice(np.array([[1, 0, 1],
                                 [0, 1, 1],
                                 [0, 0, 1]]))
test_lattice # lattice matrix를 Lattice() 객체에 넣어 격자정보를 표현한다.

Lattice
    abc : 1.4142135623730951 1.4142135623730951 1.0
 angles : 45.00000000000001 45.00000000000001 60.00000000000001
 volume : 1.0
      A : 1.0 0.0 1.0
      B : 0.0 1.0 1.0
      C : 0.0 0.0 1.0

In [ ]:
# 통합해야 하는 정보 : crystal structure에 기저(basis)와 좌표(positions) 정보를 결합해야 한다.
print(total_basis.shape)
print(total_positions.shape) # 둘이 일치해야 한다!

(50, 3, 3)
(50, 5, 3)


In [ ]:
total_crystals = []

for i in range(len(total_positions)):

    new_lattice = Lattice(total_basis[i]) # 새로운 basis
    new_frac_coords = total_positions[i]  # 새로운 fractional coordinates

    # lattice / positions 정보는 calculation history에서 추출된 정보여야 한다!!
    # 즉, 갱신할 정보는 2가지!
    each_crystal = Structure(new_lattice, crystal.species, new_frac_coords) # 기존 결정구조의 원자구성은 그대로 유지된다.
    total_crystals.append(each_crystal)

In [ ]:
# 예시로 확인
total_crystals[4] # vasprun.xml에서 positions / basis 정보 추출하여 crystal structure 구축완료

Structure Summary
Lattice
    abc : 5.973933244521023 6.050389427229451 6.008925592583448
 angles : 90.08884090101255 89.87662564570432 89.99435237241894
 volume : 217.18958168907534
      A : 5.9739145 0.00674168 -0.01336063
      B : -0.00623588 6.05038591 -0.00191703
      C : 0.02638611 -0.00738617 6.00886312
PeriodicSite: Cs (3.0101, 2.9395, 3.0179) [0.5022, 0.4859, 0.5035]
PeriodicSite: Pb (0.0156, 6.0218, 0.0111) [0.0036, 0.9953, 0.0022]
PeriodicSite: Br (5.9707, 0.0833, 2.9851) [0.9973, 0.0133, 0.4990]
PeriodicSite: Br (0.0246, 3.0339, 0.0843) [0.0046, 0.5014, 0.0142]
PeriodicSite: Br (3.0346, 0.0267, 5.9830) [0.5036, 0.0051, 0.9968]

### 4.목표 물성정보(Free energy) 추출하기


In [ ]:
e_fr_energy_list = []
e_wo_entrp_list = []
e_0_energy_list = []
count = 0

for subnode1 in root.iter('calculation'):
    for subnode2 in subnode1.findall('energy'): # iter로 하면 깊은 하위노드에 숨겨진 정보까지도 모두 보는 것 같다..
        if len(subnode2) == 3:
            for j in range(len(subnode2)):
                print(subnode2[j].tag, subnode2[j].attrib, subnode2[j].text)
                count += 1
            print('-------------------------------------------------------')
print(count)

i {'name': 'e_fr_energy'}     -16.78971335 
i {'name': 'e_wo_entrp'}     -16.78972869 
i {'name': 'e_0_energy'}       0.00004603 
-------------------------------------------------------
i {'name': 'e_fr_energy'}     -16.79068355 
i {'name': 'e_wo_entrp'}     -16.79069888 
i {'name': 'e_0_energy'}       0.00004599 
-------------------------------------------------------
i {'name': 'e_fr_energy'}     -16.79266507 
i {'name': 'e_wo_entrp'}     -16.79268036 
i {'name': 'e_0_energy'}       0.00004586 
-------------------------------------------------------
i {'name': 'e_fr_energy'}     -16.79259753 
i {'name': 'e_wo_entrp'}     -16.79261274 
i {'name': 'e_0_energy'}       0.00004562 
-------------------------------------------------------
i {'name': 'e_fr_energy'}     -16.79320367 
i {'name': 'e_wo_entrp'}     -16.79321891 
i {'name': 'e_0_energy'}       0.00004572 
-------------------------------------------------------
i {'name': 'e_fr_energy'}     -16.79403021 
i {'name': 'e_wo_entrp'}  

In [ ]:
e_fr_energy_list = []
e_wo_entrp_list = []
e_0_energy_list = []

for subnode1 in root.findall('calculation'):
    for subnode2 in subnode1.findall('energy'): # iter로 하면 깊은 하위노드에 숨겨진 정보까지도 모두 보는 것 같다..
        if len(subnode2) == 3:
            for j in range(len(subnode2)):

                if subnode2[j].attrib == {'name' : 'e_fr_energy'}:
                    e_fr_energy_list.append( float(subnode2[j].text.strip()) )
                elif subnode2[j].attrib == {'name' : 'e_wo_entrp'}:
                    e_wo_entrp_list.append( float(subnode2[j].text.strip()) )
                elif subnode2[j].attrib == {'name' : 'e_0_energy'}:
                    e_0_energy_list.append( float(subnode2[j].text.strip()) )

In [ ]:
print(e_fr_energy_list)
print('Number of Target property : {}'.format(len(e_fr_energy_list)))

[-16.78971335, -16.79068355, -16.79266507, -16.79259753, -16.79320367, -16.79403021, -16.79592155, -16.79565237, -16.79644182, -16.79644548, -16.79663639, -16.79697997, -16.79702464, -16.79724362, -16.7977269, -16.79778784, -16.79789467, -16.79805818, -16.79848412, -16.79914759, -16.80041051, -16.8014711, -16.80211373, -16.80366572, -16.80580892, -16.80749428, -16.81072843, -16.81072643, -16.81166122, -16.81320573, -16.81337528, -16.81337979, -16.81364834, -16.81415088, -16.81418722, -16.81433887, -16.81434992, -16.81441552, -16.81441569, -16.81446654, -16.81446529, -16.81449262, -16.81452226, -16.81453528, -16.81456484, -16.81458355, -16.81460321, -16.8146094, -16.81462679, -16.81464227]
Number of Target property : 50


### 5.기능을 함수로 구현



In [ ]:
def convert_POSCAR_to_structure(POSCAR_file = 'POSCAR'):
    from pymatgen.io.vasp import Poscar
    poscar = Poscar.from_file('POSCAR')
    crystal = poscar.structure
    return crystal 

In [ ]:
def check_number_of_calculation_history_from_vasprun(vasprun_xml = 'vasprun.xml'):
    calculation_history_count = 0
    for subnode1 in root.findall('calculation'):
        calculation_history_count += 1

    return calculation_history_count # 계산 history 수 확인

In [ ]:
def extract_history_basis_from_vasprun(vasprun_xml = 'vasprun.xml'):
    import re  # 정규표현식(Regular Expression)

    total_basis = []
    for subnode1 in root.iter('calculation'):
        for subnode2 in subnode1.iter('structure'):
            for subnode3 in subnode2.iter('varray'):

                if subnode3.attrib == {'name' : 'basis'}:

                    each_basis = []
                    for i in range(len(subnode3)):
                        basis = [float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)] # 정규표현식으로 숫자 변환
                        each_basis.append(basis)
                    total_basis.append(each_basis)

    total_basis = np.array(total_basis)
    return total_basis

In [ ]:
def extract_history_positions_from_vasprun(vasprun_xml = 'vasprun.xml'):
    # positions 정보에 접근하는 것에 성공했으므로 이제 전처리하여 추출한다.
    total_positions = []
    for subnode1 in root.iter('calculation'):
        for subnode2 in subnode1.iter('structure'):
            for subnode3 in subnode2.iter('varray'):
                if subnode3.attrib == {'name' : 'positions'}:
                    
                    each_positions = []
                    for i in range(len(subnode3)):
                        pos = [float(x) for x in re.findall(r"[-+]?\d*\.\d+|\d+", subnode3[i].text)]
                        each_positions.append(pos)
                    total_positions.append(each_positions)
    total_positions = np.array(total_positions)
    return total_positions

In [ ]:
def extract_free_energy_from_vasprun(vasprun_xml = 'vasprun.xml'):
    e_fr_energy_list = []
    e_wo_entrp_list = []
    e_0_energy_list = []

    for subnode1 in root.findall('calculation'): # findall??
        for subnode2 in subnode1.findall('energy'): # iter로 하면 깊은 하위노드에 숨겨진 정보까지도 모두 보는 것 같다..
            if len(subnode2) == 3:
                for j in range(len(subnode2)):

                    if subnode2[j].attrib == {'name' : 'e_fr_energy'}:
                        e_fr_energy_list.append( float(subnode2[j].text.strip()) )
                    elif subnode2[j].attrib == {'name' : 'e_wo_entrp'}:
                        e_wo_entrp_list.append( float(subnode2[j].text.strip()) )
                    elif subnode2[j].attrib == {'name' : 'e_0_energy'}:
                        e_0_energy_list.append( float(subnode2[j].text.strip()) )
    return e_fr_energy_list

In [ ]:
# Function Check
crystal = convert_POSCAR_to_structure('POSCAR')
crystal_basis = extract_history_basis_from_vasprun()
crystal_positions = extract_history_positions_from_vasprun()
crystal_free_energys = extract_free_energy_from_vasprun()
history_number = check_number_of_calculation_history_from_vasprun()
print(history_number)
print(len(crystal_free_energys)) # 일치해야 함!

50
50


### XML file 전체 Parsing

1. 개별 sub directory에 대한 XML file 전처리가 가능했으므로 이제 이것을 전체 디렉터리에 대해 일반화한다.
2. 모든 계산 디렉터리를 순회하며, 개별 디렉터리에서 했던 전처리 과정을 반복하고 데이터셋을 정리하면 3600개의 Nonlocal Halide Perovskite 데이터셋을 얻을 수 있다.



In [ ]:
%cd /content/drive/My Drive/Hyeongseon Park Research/Halide Perovskite Dataset

/content/drive/My Drive/Hyeongseon Park Research/Halide Perovskite Dataset


In [ ]:
import os
import glob
from tqdm import tqdm

In [ ]:
path = './*'
file_list = glob.glob(path)
dir_list = [element for element in file_list if os.path.isdir(element)] # 계산 디렉터리 리스트 추출
dir_list_sorted = sorted(dir_list) # 정렬

In [ ]:
dir_list_sorted[-1] # 3600

'./3600_CsSnI3_mp-614013_r2xr2x2_with_noise_99'

In [ ]:
N = 50
def get_exponential_index(N):
    for i in range(10):
        if 2**i >= N:
            return i

indexes = [0]
for j in range(get_exponential_index(N)):
    indexes.append(2**j)
indexes.append(-1)

indexes

[0, 1, 2, 4, 8, 16, 32, -1]

In [ ]:
total_crystal_dataset = []
total_energy_dataset = []

for each_dir in tqdm(dir_list_sorted):

    os.chdir(each_dir) # Change directory

    try:
        # 개별 디렉터리 위치에서 하나의 vasprun.xml, POSCAR 파일에 대해 전처리 시도
        xml_file = './vasprun.xml'
        poscar_file = './POSCAR'

        doc = ET.parse(xml_file) # XML Parsing
        root = doc.getroot()     # root node

        # Five Preprocessing Function activated
        crystal         = convert_POSCAR_to_structure(poscar_file)
        num_history     = check_number_of_calculation_history_from_vasprun()
        total_basis     = extract_history_basis_from_vasprun()
        total_positions = extract_history_positions_from_vasprun()
        free_energys    = extract_free_energy_from_vasprun() 
        free_energys    = list ( np.array(free_energys) / len(crystal) ) # 원자수로 나누어 Energy_per_atom으로 물성변환

        # 통합
        total_crystals = []
        total_energies = []

        # Exponential 꼴로 index 모은다.
        indexes = [0]
        for j in range(get_exponential_index(N)):
            indexes.append(2**j)
        indexes.append(-1)

        for i in indexes: # 0, 1, 2, 4, 8, 16, ..., -1(마지막) history 정보만 선택

            new_lattice = Lattice(total_basis[i])
            new_frac_coords = total_positions[i]
            free_energy = free_energys[i]

            # Lattice / positions 정보는 calculation history의 정보여야만 한다.
            # 즉, 갱신할 정보는 2가지!
            each_crystal = Structure(new_lattice, crystal.species, new_frac_coords) # 결정구조의 원자는 그대로

            total_crystals.append(each_crystal)
            total_energies.append(free_energy)

        total_crystal_dataset.extend(total_crystals)
        total_energy_dataset.extend(total_energies)

    except Exception as e:  # 계산되지 않은 디렉터리가 있거나 추가적인 에러가 발생할 경우
        print(e, each_dir)  # 에러와 에러발생한 디렉터리 위치 출력

    finally:
        os.chdir('../') # Change directory to previous directory

print('-------------------- Total XML Parsing Completed!! --------------------')

 51%|█████     | 1827/3600 [01:52<01:11, 24.82it/s]

no element found: line 5402, column 0 ./1824_CsSnBr3_mp-27214_1x1x1_with_noise_3


 64%|██████▎   | 2289/3600 [02:19<00:46, 28.42it/s]

not well-formed (invalid token): line 901, column 0 ./2282_CsSnBr3_mp-27214_r2xr2x1_with_noise_82


100%|██████████| 3600/3600 [03:36<00:00, 16.64it/s]

-------------------- Total XML Parsing Completed!! --------------------


In [ ]:
# check
each_dir # 3600

'./3600_CsSnI3_mp-614013_r2xr2x2_with_noise_99'

In [ ]:
print(len(total_crystal_dataset))
print(len(total_energy_dataset)) # 일치해야 함

28784
28784


In [ ]:
total_crystal_dataset[2] # structure

Structure Summary
Lattice
    abc : 5.993071317362744 6.034392557198279 6.0056034784538115
 angles : 90.03170441456155 89.90463608204185 89.99458701755403
 volume : 217.1895818849653
      A : 5.99304902 0.00674921 -0.01488983
      B : -0.00622296 6.03438925 0.00109024
      C : 0.02492167 -0.00438251 6.00555017
PeriodicSite: Cs (3.0182, 2.9387, 3.0186) [0.5020, 0.4868, 0.5038]
PeriodicSite: Pb (0.0194, 5.9888, 0.0168) [0.0043, 0.9924, 0.0026]
PeriodicSite: Br (5.9872, 0.0864, 2.9738) [0.9970, 0.0136, 0.4976]
PeriodicSite: Br (0.0237, 3.0349, 0.0881) [0.0044, 0.5029, 0.0146]
PeriodicSite: Br (3.0426, 0.0304, 5.9803) [0.5036, 0.0052, 0.9970]

In [ ]:
# 계산 히스토리 보기
for i in range(len(total_crystal_dataset)):
    print(len(total_crystal_dataset[i])) 

Streaming output truncated to the last 5000 lines.
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5

In [ ]:
new_id_prop_data = []
for each_crystal, each_energy in zip(total_crystal_dataset, total_energy_dataset):
    each_data = [each_crystal, each_energy]
    new_id_prop_data.append(each_data)

In [ ]:
# Check
idx = 2
crystal, target = new_id_prop_data[idx]
print(target)
crystal # 여기까지 하면 일단 데이터 전처리는 끝!

-3.3585330140000003


Structure Summary
Lattice
    abc : 5.993071317362744 6.034392557198279 6.0056034784538115
 angles : 90.03170441456155 89.90463608204185 89.99458701755403
 volume : 217.1895818849653
      A : 5.99304902 0.00674921 -0.01488983
      B : -0.00622296 6.03438925 0.00109024
      C : 0.02492167 -0.00438251 6.00555017
PeriodicSite: Cs (3.0182, 2.9387, 3.0186) [0.5020, 0.4868, 0.5038]
PeriodicSite: Pb (0.0194, 5.9888, 0.0168) [0.0043, 0.9924, 0.0026]
PeriodicSite: Br (5.9872, 0.0864, 2.9738) [0.9970, 0.0136, 0.4976]
PeriodicSite: Br (0.0237, 3.0349, 0.0881) [0.0044, 0.5029, 0.0146]
PeriodicSite: Br (3.0426, 0.0304, 5.9803) [0.5036, 0.0052, 0.9970]

In [ ]:
# Save Data for reuse
import pickle
with open('Nonlocal-Halide-Perovskite-Dataset.pickle', 'wb') as fw:
    pickle.dump(new_id_prop_data, fw)

In [ ]:
# Load Data
with open('Nonlocal-Halide-Perovskite-Dataset.pickle', 'rb') as fr:
    new_id_prop_data = pickle.load(fr)

In [ ]:
# Check
idx = 2
crystal, target = new_id_prop_data[idx]
print(target)
crystal # 여기까지 하면 일단 데이터 전처리는 끝!

-3.3585330140000003


Structure Summary
Lattice
    abc : 5.993071317362744 6.034392557198279 6.0056034784538115
 angles : 90.03170441456155 89.90463608204185 89.99458701755403
 volume : 217.1895818849653
      A : 5.99304902 0.00674921 -0.01488983
      B : -0.00622296 6.03438925 0.00109024
      C : 0.02492167 -0.00438251 6.00555017
PeriodicSite: Cs (3.0182, 2.9387, 3.0186) [0.5020, 0.4868, 0.5038]
PeriodicSite: Pb (0.0194, 5.9888, 0.0168) [0.0043, 0.9924, 0.0026]
PeriodicSite: Br (5.9872, 0.0864, 2.9738) [0.9970, 0.0136, 0.4976]
PeriodicSite: Br (0.0237, 3.0349, 0.0881) [0.0044, 0.5029, 0.0146]
PeriodicSite: Br (3.0426, 0.0304, 5.9803) [0.5036, 0.0052, 0.9970]

In [ ]:
# 현재 위치에는 3600개의 계산디렉토리가 있으므로 다른 위치로 pickle 데이터를 옮겨준다.
# !mv Nonlocal-Halide-Perovskite-Dataset.pickle ../

## 전처리 완료!